In [ ]:
def multi(imp_fert):
    imp_fert.date=imp_fert.date.astype('datetime64[ns]')
    
    date_cols=['CO_ANO','CO_MES']
    cat_cols=['CO_NCM','SG_UF_NCM']
    quanti_cols=['KG_LIQUIDO','VL_FOB']
    
    # Cria colunas com valores adiantados
    ad=1
    imp_fert_ad=imp_fert.set_index('date').shift(1,freq='ME').shift(ad,freq='D').reset_index()
    imp_fert_ad=imp_fert_ad.rename(columns={'KG_LIQUIDO':'kg_ad_'+str(ad),'VL_FOB':'fob_ad_'+str(ad)})
    
    dates_to_drop=imp_fert_ad.iloc[ad*(-1):,:].date.values
    
    imp_fert_ad=imp_fert_ad[~imp_fert_ad.date.isin(dates_to_drop)]
    
    # Divide em duas tabelas
    imp_fert_ad_kg=imp_fert_ad[['date']+cat_cols+['kg_ad_'+str(ad)]]
    imp_fert_ad_fob=imp_fert_ad[['date']+cat_cols+['fob_ad_'+str(ad)]]
    
    # Unifica categorias
    imp_fert_ad_kg.insert(0,'cat_cols',imp_fert_ad.CO_NCM+'~'+imp_fert_ad.SG_UF_NCM+'_kg_ad_'+str(ad))
    imp_fert_ad_fob.insert(0,'cat_cols',imp_fert_ad.CO_NCM+'~'+imp_fert_ad.SG_UF_NCM+'_fob_ad_'+str(ad))
    
    # Descarta categorias antigas
    imp_fert_ad_kg=imp_fert_ad_kg.drop(columns=cat_cols)
    imp_fert_ad_fob=imp_fert_ad_fob.drop(columns=cat_cols)
    
    def func(tab_e):
        tab_s=pd.DataFrame()
        for date in tab_e.date.unique():
            tab_e_t=tab_e[tab_e.date==date].drop(columns='date').set_index('cat_cols').transpose()\
            .assign(date=date).reset_index(drop=True)
            tab_s=pd.concat([tab_s,tab_e_t])
        return tab_s
    
    imp_fert_ad_kg_w=func(imp_fert_ad_kg)
    
    imp_fert_ad_fob_w=func(imp_fert_ad_fob)
    
    imp_fert_ad_w=imp_fert_ad_kg_w.merge(imp_fert_ad_fob_w)
    
    imp_fert=imp_fert.merge(imp_fert_ad_w)
    
    return imp_fert

In [ ]:
print('*** multi(imp_fert) function available ***')